In [1]:
%pip install scipy==1.8.0 numpy==1.21.5
from rosbag2_reader_py import Rosbag2Reader
from Gaussian_Filters import utils


Note: you may need to restart the kernel to use updated packages.


## Open ROS 2 Bag

**The file `rosbag2_reader_py.py` shall be in the same folder of the notebook.**

In [2]:
path = "/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation"

reader = Rosbag2Reader(path)
topics = reader.all_topics
topics

[INFO] [1765879952.418611891] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.
[INFO] [1765879952.419231596] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.


{'/ekf': 'nav_msgs/msg/Odometry',
 '/ground_truth': 'nav_msgs/msg/Odometry',
 '/odom': 'nav_msgs/msg/Odometry'}

## Get total number of messages in the bag

In [3]:
tot_msgs = 0
for _ in reader:
    tot_msgs += 1

print(f"Total messages: {tot_msgs}")

[INFO] [1765879952.431738913] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.


Total messages: 3347


## Select messages of specific topics

In [4]:
tot_msgs = 0
reader.set_filter(["/odom"])
for _ in reader:
    tot_msgs += 1

print("After the filter is applyed: ", reader.selected_topics)
print(f"Total messages: {tot_msgs}")

reader.reset_filter() # if you want to read all messages after you set a filter
print("After the filter is reset: ", reader.selected_topics)

After the filter is applyed:  {'/odom': 'nav_msgs/msg/Odometry'}
Total messages: 1808
After the filter is reset:  {'/ekf': 'nav_msgs/msg/Odometry', '/ground_truth': 'nav_msgs/msg/Odometry', '/odom': 'nav_msgs/msg/Odometry'}


[INFO] [1765879952.886400304] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.
[INFO] [1765879952.887079956] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.
[INFO] [1765879953.104344880] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.


## Access only data of a given type

In this example you can see  how to access an `Odometry` message checking for its type.

Please, notice the difference between **recording time** and time reported in the **stamp**. This is because the message was generated at a time that does not coincide with the time the message was received and recorded. This difference in a real robot may be really small, in the order of microseconds, but for a simulation, as in the reported case, the time could be extremely different. **You shall always use `header.stamp` whenever it is available.**

In [5]:
from rclpy.time import Time
from nav_msgs.msg import Odometry
for topic_name, msg, t in reader:
    print(f"Received message of type {type(msg).__name__} on topic {topic_name} recorded at time {t}")
    if type(msg) is Odometry:
        time = Time.from_msg(msg.header.stamp).nanoseconds
        print(f"Position (x, y) at time {time}: ({msg.pose.pose.position.x:.2f}, {msg.pose.pose.position.y:.2f})")
    break

Received message of type Odometry on topic /ekf recorded at time 1764274742685079435
Position (x, y) at time 1764274742684655635: (-1.99, -0.49)


[INFO] [1765879953.125521484] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.


## Interpolate data to compute metrics

In order to compute the metrics for `/odom` and `/ekf` topics, you have to compare the poses reported in these topic with the poses reported in topic `/ground_truth` in the same time instants. 

Since the data are generated from different nodes at different frequencies, the time of the various topics will be different. So, we need to interpolate ground truth data on the time scale of the topic we want to evaluate.

First of all, let us save relevant data from messages in some NumPy arrays. As you can see from the output, the number of points from the two topics is different.

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

time_gt = []
gt_data = []
time_odom = []
odom_data = []

reader.set_filter(["/ground_truth", "/ekf"])

for topic_name, msg, t in reader:
    if topic_name == "/ground_truth":
        time_gt.append(Time.from_msg(msg.header.stamp).nanoseconds)
        gt_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/ekf":
        time_odom.append(Time.from_msg(msg.header.stamp).nanoseconds)
        odom_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))

time_gt = np.array(time_gt)
gt_data = np.array(gt_data)
time_odom = np.array(time_odom)
odom_data = np.array(odom_data)

print(f"Ground truth points: {len(gt_data)}")
print(f"Odometry points: {len(odom_data)}")
odom_data
#time_gt, time_odom

[INFO] [1765879953.305116976] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.
[INFO] [1765879953.305777823] [rosbag2_storage]: Opened database '/home/marco/HELP/ERROR PLOTS/Gazebo rosbag simulation/rosbag2_2025_11_27-21_19_02_0.db3' for READ_ONLY.


Ground truth points: 1229
Odometry points: 310


(array([266881000000, 266931000000, 266981000000, ..., 328181000000,
        328231000000, 328281000000]),
 array([1764274742684655635, 1764274742885388599, 1764274743085993558,
        1764274743285598009, 1764274743485967161, 1764274743685727912,
        1764274743886342563, 1764274744087245509, 1764274744287009247,
        1764274744486492486, 1764274744687139324, 1764274744887622662,
        1764274745088601895, 1764274745288272721, 1764274745488914447,
        1764274745688030373, 1764274745889007298, 1764274746089654118,
        1764274746288479132, 1764274746489952844, 1764274746690080058,
        1764274746889804371, 1764274747089445779, 1764274747292366758,
        1764274747490706749, 1764274747691047035, 1764274747891126622,
        1764274748090669605, 1764274748290677780, 1764274748492233252,
        1764274748691805028, 1764274748893050900, 1764274749092300471,
        1764274749291705635, 1764274749493421994, 1764274749694694053,
        1764274749893792038, 176427475009

Now, let us create an interpolating function using SciPy `interp1d`.

In [7]:
#gt_interpol = interp1d(time_gt, gt_data, axis=0,fill_value='extrapolate', kind="nearest")
#gt_data_interp = gt_interpol(time_odom)
#print(f"Interpolated ground truth points: {len(gt_data_interp)}")
#gt_data_interp

import numpy as np
from scipy.interpolate import interp1d

# 1. Normalize both time arrays to start at 0.0 seconds
# Subtract the first timestamp from every element to align the starts.
# Divide by 1e9 to convert Nanoseconds -> Seconds (crucial for precision).
t0_gt = time_gt[0]
t0_odom = time_odom[0]

time_gt_sec = (time_gt - t0_gt) / 1e9
time_odom_sec = (time_odom - t0_odom) / 1e9

# 2. Create the interpolator using the NORMALIZED Ground Truth time
gt_interpol = interp1d(time_gt_sec, gt_data, axis=0, 
                       fill_value='extrapolate', kind="nearest")

# 3. Interpolate using the NORMALIZED Odometry time
# Now that both arrays share the same 0-based timeline, this will work.
gt_data_interp = gt_interpol(time_odom_sec)

# 4. Verify results
print(f"Interpolated ground truth points: {len(gt_data_interp)}")
gt_data_interp

Interpolated ground truth points: 310


array([[-1.99992652e+00, -5.00001021e-01],
       [-1.99992653e+00, -5.00001021e-01],
       [-1.99992651e+00, -5.00001021e-01],
       [-1.99992647e+00, -5.00001021e-01],
       [-1.99992652e+00, -5.00001021e-01],
       [-1.99992643e+00, -5.00001021e-01],
       [-1.99992646e+00, -5.00001021e-01],
       [-1.99992647e+00, -5.00001021e-01],
       [-1.99992641e+00, -5.00001021e-01],
       [-1.99992645e+00, -5.00001021e-01],
       [-1.99992637e+00, -5.00001021e-01],
       [-1.99987678e+00, -5.00001021e-01],
       [-1.99833742e+00, -5.00001016e-01],
       [-1.99633651e+00, -5.00001010e-01],
       [-1.99433639e+00, -5.00001005e-01],
       [-1.99233701e+00, -5.00001000e-01],
       [-1.99004674e+00, -5.00000994e-01],
       [-1.98582797e+00, -5.00000982e-01],
       [-1.97989765e+00, -5.00000962e-01],
       [-1.97389620e+00, -5.00000944e-01],
       [-1.96789745e+00, -5.00000927e-01],
       [-1.96189713e+00, -5.00000909e-01],
       [-1.95589651e+00, -5.00000892e-01],
       [-1.

Compute Mean Absolute Error between odometry data and interpolated ground truth. You can find already implemented metrics functionson Portale della Didattica (Lecture_notebooks/Gaussian_filters.zip/utils.py)

In [8]:
error=np.linalg.norm(odom_data - gt_data_interp, axis=1)
MAE  = np.mean(error)
RMSE = np.sqrt(np.mean(error**2))

print("MAE:", MAE)
print("RMSE:", RMSE)


MAE: 0.0754468068337693
RMSE: 0.08304609069697781
